In [ ]:
# !pip install dotenv
# !pip install paramiko
# !pip install fabric
# !pip install pytz

In [45]:
import os
from dotenv import load_dotenv
import subprocess
import re
import pytz
from datetime import datetime
from fabric import Connection
import requests
from pathlib import Path


load_dotenv()

True

In [25]:
# Load secrets from .env
SECRET_KEY = os.getenv("SECRET_KEY")
IMMICH_API_KEY = os.getenv("IMMICH_API_KEY")
IMMICH_UPLOAD_URL = str(os.getenv("IMMICH_UPLOAD_URL"))
SERVER_ADDRESS = str(os.getenv("SERVER_ADDRESS"))
SERVER_USERNAME = str(os.getenv("SERVER_USERNAME"))
SERVER_PASSWORD = str(os.getenv("SERVER_PASSWORD"))
# SSH_KEY_LOC = os.getenv("SSH_KEY_LOC")

TMP_DIR = "./tmp"

In [ ]:

# Create a connection using password authentication
conn = Connection(
    host="192.168.0.61",
    user="jstephencorey",
    connect_kwargs={
        "password": SERVER_PASSWORD
    }
)

# Verify it works
result = conn.run("hostname", hide=True)
print("Connected to:", result.stdout.strip())


Connected to: jserver


In [48]:
remote_root = f"/zstorage/surveillance_video"

video_name = "20251022_165042_clip_03709_1.mp4"

remote_filepath = f"{remote_root}/{video_name}"
local_filepath = f"{TMP_DIR}/{video_name}"

# Download (copy)
conn.get(remote_filepath, local_filepath)

# Delete the original (move)
conn.run(f"rm {remote_filepath}")

print("File moved successfully!")


File moved successfully!


In [49]:
av1_filename = video_name.split(".")[0] + ".mkv"
av1_filepath = f"{TMP_DIR}/{av1_filename}"
assert not os.path.exists(av1_filepath)
HIGH_QUALITY = "30"
LOW_QUALITY = "37" #Found through testing to be a good balance of size and not losing fidelity

cmd = [
    "ffmpeg",
    "-hwaccel", "cuda",
    "-i", local_filepath,
    "-c:v", "av1_nvenc",
    "-preset", 'p7',
    "-cq", LOW_QUALITY,
    "-c:a", "copy",
    av1_filepath
]

subprocess.run(cmd, check=True)

CompletedProcess(args=['ffmpeg', '-hwaccel', 'cuda', '-i', './tmp/20251022_165042_clip_03709_1.mp4', '-c:v', 'av1_nvenc', '-preset', 'p7', '-cq', '37', '-c:a', 'copy', './tmp/20251022_165042_clip_03709_1.mkv'], returncode=0)

In [50]:
def get_iso8601_zoned_timestamp():
    tz = pytz.timezone(os.getenv("TIMEZONE", "America/Denver"))  # fallback to Mountain Time
    now = datetime.now(tz)
    return now.isoformat()

def extract_iso8601_from_filename(filename):
    match = re.search(r"(\d{8}_\d{6})", filename)
    if not match:
        print("NOTE: BAD DATE EXTRACTION STUFF, RETURNING CURRENT TIME")
        return get_iso8601_zoned_timestamp()
    timestamp_str = match.group(1)
    tz = pytz.timezone(os.getenv("TIMEZONE", "America/Denver"))
    dt = datetime.strptime(timestamp_str, "%Y%m%d_%H%M%S")
    dt_local = tz.localize(dt)
    return dt_local.isoformat()

timestamp = "20251023_212137"

extract_iso8601_from_filename(timestamp)
# get_iso8601_zoned_timestamp()

'2025-10-23T21:21:37-06:00'

In [52]:
DEVICE_ID = "living_room_rpi_cam"

def send_image_to_immich(video_filepath):
    try:
        # Expect deviceId in form data
        print("beginning to process the request")
        video_filename = Path(video_filepath).name
        print(video_filename)
        stats = os.stat(video_filepath)
        with open(video_filepath, 'rb') as f:
            headers = {'x-api-key': IMMICH_API_KEY}
            tz = pytz.timezone(os.getenv("TIMEZONE", "America/Denver"))
            files = {'assetData': f}
            file_modified_at_timestamp = datetime.fromtimestamp(stats.st_mtime, tz).isoformat()
            file_created_at_timestamp = datetime.fromtimestamp(stats.st_ctime, tz).isoformat()
            print(file_modified_at_timestamp)

            data = {
                'deviceId': DEVICE_ID,
                'deviceAssetId': video_filename,
                'fileCreatedAt': file_created_at_timestamp,
                'fileModifiedAt': file_modified_at_timestamp
            }
            print(f"Uploading video to Immich with deviceAssetId: {video_filename}")
            response = requests.post(IMMICH_UPLOAD_URL, files=files, data=data, headers=headers)
            response.raise_for_status()
            print(f"Upload response code: {response.status_code}")
            print(response.json())
            if response.json()['status'] != 'created':
                print("Something went wrong!")
                raise RuntimeError()
    #     return {"status": "success", "immich_response": response.json()}, response.status_code

    except Exception as e:
        print("Error during send_image_to_immich")
        
send_image_to_immich(av1_filepath)

beginning to process the request
20251022_165042_clip_03709_1.mkv
2025-10-23T22:07:13.411836-06:00
Uploading video to Immich with deviceAssetId: 20251022_165042_clip_03709_1.mkv
Upload response code: 200
{'status': 'duplicate', 'id': '6ffb4546-95ff-4c7d-90c9-e828b5e208af'}
Something went wrong!
Error during send_image_to_immich
